In [1]:
import os
import re
import matplotlib
import pandas
import numpy
from sympy import Point, Line, Segment
from numpy import sqrt
from scipy import stats
import matplotlib.pyplot as plt


# list all files 
def find_all_files(folder, ext, func = str.endswith):
    files = [f for f in os.listdir(folder) if func(f, ext)]
    return files

def output_folder_name():
    now = datetime.datetime.now()
    return 'output_' + now.strftime("%Y%m%d%H%m")

def enumerate_folder(parent):
    folders = []
    for f in os.listdir(parent):
        f = os.path.join(parent, f)
        if os.path.isdir(f):
            folders.append(f)
    return folders


def reform_file_name(filename, fm, to) :
    return filename.replace(fm, to)

def change_file_name(folder) :
#    files = os.listdir(folder)
    files = find_all_files(folder, '.csv')
    for f in files:
        nf = reform_file_name(f, '_', '-')
        if f != nf:
            print f, nf
        os.rename(os.path.join(folder, f),  os.path.join(folder, nf))

def convert_delimiter(filename, fm, to) :
    with open(filename, 'r') as f:
        lines = f.readlines()
        nl = [to.join(line.split(fm)) for line in lines]

    with open(filename, 'w') as f:
        f.writelines(nl)
        
def convert_file_delimiter(folder) :
    files = find_all_files(folder, '.csv')
    for f in files:
        convert_delimiter(os.path.join(folder, f), '\t', ',')
        
def load_zip_file(filename):
    import zipfile
    zip_ref = zipfile.ZipFile(filename, 'r')
    zip_ref.extractall(os.getcwd())
    zip_ref.close()
    return os.path.join(os.getcwd(), os.path.splitext(filename)[0])

# read from csv file and get x,y values of points
def read_file(filename, usecols = ['X', 'Y']) :
    df = pandas.read_csv(filename, skiprows = 0, sep=',', usecols=usecols)
    df.dropna(how="all", inplace=True)
    return df

# check if projection of a point to line is between two points
def is_projection_between_two_points(p, p1, p2) :
    #return (p1.x - p.x)*(p2.x - p.x) <= 0 and (p1.y - p.y) * (p2.y - p.y) <= 0
    seg = Segment(p1, p2)
    return seg.contains(p)

# group x,y into a point object
def group_to_points(data) :
    return [Point(v, data['Y'][i], evaluate=False) for i,v in enumerate(data['X'])]

# compute distance between two distance:
def subtraction(points) :
    return [points[i].distance(points[i-1]) for i in range(1, len(points))]

def make_file_pairs(files) :
    pairs = []
    pattern = re.compile(r'\w+(-\d+){3,4}.csv')
    #pattern = re.compile(r'\w+.csv')
    for f in files :
        if pattern.match(f):
            another = f[:f.index('.csv')] + 's.csv'
            pairs.append((f, another))
    return pairs

def compute(p1, p2) :
    s1 = subtraction(p1)
    s2 = subtraction(p2)
    d1, li = compute_distances_between_two_points_set(p1, p2, check_segment_crossing)
    d2, li = compute_distances_between_two_points_set(p2, p1, check_segment_crossing)
    return s1 + s2, d1 + d2

def output_csv(data, filename) :
    pdata = pandas.DataFrame(data)
    pdata.to_csv(filename)

def points_from_pairs(sub = ""):
    folder = os.getcwd()
    if sub != "":
        folder = os.path.join(folder, sub)
    files = find_all_files(folder, '.csv')
    pairs = make_file_pairs(files)
    for pair in pairs:
        f1, f2 = pair[0], pair[1]
        d1 = read_file(os.path.join(folder, f1))
        d2 = read_file(os.path.join(folder, f2))
        p1 = group_to_points(d1)
        p2 = group_to_points(d2)
        yield (f1,) + compute(p1, p2)

def points_from_single_file(sub = "") :
    folder = os.getcwd()
    if sub != "":
        folder = os.path.join(folder, sub)
    files = find_all_files(folder, '.csv')
    for f in files:
        d = read_file(os.path.join(folder, f))
        p = group_to_points(d)
        p1 = [v for i,v in enumerate(p) if not (i % 2)]
        p2 = [v for i,v in enumerate(p) if i % 2]
        yield (f,) +  compute(p1, p2)

def check_segment_crossing(p1,p2,q1,q2) :
    p = Segment(p1, p2)
    q = Segment(q1, q2)
    return len(p.intersection(q)) > 0

        
# compute distances between point and line in two points set
def compute_distances_between_two_points_set(points1, points2, check = None) :
    index = 0
    distances = []
    i = 0
    li = []
    while i < len(points1):
        p1 = points1[i]
        if index >= len(points2) - 1: break

        if check:
            if i < len(points1) - 1:
                if check(points2[index], points2[index+1], points1[i], points1[i+1]):
                    print "crossing~"

        line = Line(points2[index], points2[index + 1])
        pro = line.projection(p1)
        
        if is_projection_between_two_points(pro, points2[index], points2[index + 1]):
            distances.append(p1.distance(pro))
            li.append([p1, pro])
            index += 1
        else:
            if pro.distance(points2[index + 1]) < pro.distance(points2[index]):
                i -= 1
                index += 1
        i += 1
    return distances, li

def run(sub, output, pair = True):
    out_l = {'file':[], 'length':[]}
    out_d = {'file':[], 'dist':[]}
    if pair:
        results = points_from_pairs(sub)
    else:
        results = points_from_single_file(sub)
    for f, l, d in results:
        out_l['file'] += [f for i in l]
        out_l['length'] += l
        out_d['file'] += [f for i in d]
        out_d['dist'] += d

    output_csv(out_l, os.path.join(output, os.path.basename(sub) + '_output_length.csv'))
    output_csv(out_d, os.path.join(output, os.path.basename(sub) + '_output_dist.csv'))

def test_image(folder, filename, pair = True):
    if pair:
        f1, f2 = os.path.join(folder, filename + '.csv'), os.path.join(folder, filename+'s.csv')
        d1 = read_file(os.path.join(folder, f1))
        d2 = read_file(os.path.join(folder, f2))
        p1 = group_to_points(d1)
        p2 = group_to_points(d2)
    else:
        d = read_file(os.path.join(folder, filename + '.csv'))
        p = group_to_points(d)
        p1 = [v for i,v in enumerate(p) if i % 2 == 0]
        p2 = [v for i,v in enumerate(p) if i % 2 != 0]
    plt.figure()
    x = [p.x for p in p1]
    y = [p.y for p in p1]
    plt.plot(x, y, c='r', marker='*')
    x = [p.x for p in p2]
    y = [p.y for p in p2]
    plt.plot(x, y, c='r', marker='.')
    d1,li = compute_distances_between_two_points_set(p1, p2)
    for i,l in enumerate(li):
        x = [p.x for p in l]
        y = [p.y for p in l]
        plt.plot(x, y, c='g', label=str(d1[i]))

    d2, li = compute_distances_between_two_points_set(p2, p1)
    for i,l in enumerate(li):
        x = [p.x for p in l]
        y = [p.y for p in l]
        plt.plot(x, y, c='b', label=str(d2[i]))

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(filename)
    plt.gca().set_aspect('equal', adjustable='box')
    #plt.show()

    ax=plt.gca()                                # get the axis
    ax.set_ylim(ax.get_ylim()[::-1])
    plt.savefig(os.path.join(folder, filename+'.png'))
    plt.close()

def generate_50_min_variance(data) :
    ndata = numpy.array(data)
    mean = ndata.mean()
    top = [(i, abs(i-mean)) for i in data]
    top = sorted(top, key=lambda i:i[1])
    return [i[0] for i in top[:50]]


def generate_50_random(data) :
    return numpy.random.choice(data, 50)


def reform_output(sub, generate_function) :
    folder = os.path.join(os.getcwd(),sub)
    files = find_all_files(folder, '.csv')
    for f in files:
        keyword = os.path.splitext(f)[0].split('_')[-1]
        data = read_file(os.path.join(folder,f), usecols=['file', keyword])
        names = data['file']
        values = data.get(keyword)
        buffer = {}
        for i, name in enumerate(names) :
            tag = '-'.join(name.split('-')[:2])
            buffer.setdefault(tag, []).append(values[i])

        output_data = {}
        for k,v in buffer.items():
            if len(v) <= 50:
                output_data[k] = v
                print len(v), "<= 50", f, k
                continue
            output_data[k] = generate_function(v)

        postfix = '_selection.csv' if select_function == 'closest' else '_random_selection.csv'
        with open(os.path.join(folder, os.path.splitext(f)[0] + postfix), 'w') as fout:
            fout.write('tag,value\n')
            for k,v in output_data.items():
                for i in v:
                    fout.write(k)
                    fout.write(',')
                    fout.write(str(i))
                    fout.write('\n')

def show_histogram(csv) :
    data = pandas.read_csv(csv)
    plt.figure()
    plt.hist(data['value'], bins=50)
    plt.title(os.path.basename(csv))
    ndata = numpy.array(data['value'])
    print ndata.mean()
    print ndata.var()
    plt.savefig(csv +'.png')
    plt.close()

    
# output images
def output_all_image(folder, pair = True):
    files = find_all_files(folder, '.csv')
    if pair:
        pairs = make_file_pairs(files)
        for pair in pairs:
            test_image(folder, os.path.splitext(pair[0])[0], pair)
    else:
        for f in files:
            test_image(folder, os.path.splitext(f)[0], pair)

def plot_image(sub):
    folder = os.getcwd()
    folder = os.path.join(folder, sub)
    files = find_all_files(folder, '_selection.csv', str.endswith)
    print files
    for f in files:
        show_histogram(os.path.join(folder, f))


In [2]:
# **********     all start from here    *****************
# ********  define all the parameters here     **********

# data with zip format
data_zip_file = 'unc-31_mec10_move.zip'
# whether use pair data like one is "wt-1-1-1.csv" the pair data is "wt-1-1-1s.csv"
use_data_pair = False
# output folder
output_folder_name = '_output' # output_folder_name()
# select function
select_function = 'random'

In [3]:
# step 1: load data
data_folder = load_zip_file(data_zip_file)
sub_folders = enumerate_folder(data_folder)
output_folder = data_folder + output_folder_name

In [4]:
# step 2: regulate file names
for folder in sub_folders:
    change_file_name(folder)

wy9007_3_3_1.csv wy9007-3-3-1.csv
wy9007_1_5_1.csv wy9007-1-5-1.csv
wy9007_8_2_1.csv wy9007-8-2-1.csv
wy9007_5_4_1.csv wy9007-5-4-1.csv
wy9007_10_4_1.csv wy9007-10-4-1.csv
wy9007_8_3_1.csv wy9007-8-3-1.csv
wy9007_5_1_1.csv wy9007-5-1-1.csv
wy9007_6_3_2.csv wy9007-6-3-2.csv
wy9007_4_1_1.csv wy9007-4-1-1.csv
wy9007_6_121.csv wy9007-6-121.csv
wy9007_7_3_2.csv wy9007-7-3-2.csv
wy9007_10_3_1.csv wy9007-10-3-1.csv
wy9007_5_3_2.csv wy9007-5-3-2.csv
wy9007_4_2_1.csv wy9007-4-2-1.csv
wy9007_8_1_1.csv wy9007-8-1-1.csv
wy9007_4_4_1.csv wy9007-4-4-1.csv
wy9007_4_3_1.csv wy9007-4-3-1.csv
wy9007_10_2_1.csv wy9007-10-2-1.csv
wy9007_7_3_3.csv wy9007-7-3-3.csv
wy9007_7_1_1.csv wy9007-7-1-1.csv
wy9007_6_2_2.csv wy9007-6-2-2.csv
wy9007_8_2_2.csv wy9007-8-2-2.csv
wy9007_7_3_1.csv wy9007-7-3-1.csv
wy9007_3_1_1.csv wy9007-3-1-1.csv
wy9007_5_2_1.csv wy9007-5-2-1.csv
wy9007_6_1_1.csv wy9007-6-1-1.csv
wy9007_2_3_2.csv wy9007-2-3-2.csv
wy9007_3_4_1.csv wy9007-3-4-1.csv
wy9007_2_2_1.csv wy9007-2-2-1.csv
wy9007_5

In [5]:
# step 3: change delimiter from '\t' to ','
for folder in sub_folders:
    convert_file_delimiter(folder)

In [6]:
# step 4: compute distances
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
for folder in sub_folders:
    sub_output_folder = os.path.join(output_folder, os.path.split(folder)[-1])
    if not os.path.exists(sub_output_folder):
        os.mkdir(sub_output_folder)
    run(folder, sub_output_folder, use_data_pair)

crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~
crossing~


In [ ]:
# step 5: output c-elegans images
for folder in sub_folders:
    output_all_image(folder, use_data_pair)

In [16]:
# step 6: selection and output distributed image
all_output_folders = enumerate_folder（output_folder)
for output in all_output_folders:
    if select_function == 'random':
        reform_output(output, generate_50_random)
    elif select_function == 'closest':
        reform_output(output, generate_50_min_variance)
    else:
        print "select_function: %s not found" % select_function

48 <= 50 N2_output_dist.csv wt-8
49 <= 50 N2_output_dist.csv wt-6
43 <= 50 N2_output_dist.csv wt-5
49 <= 50 N2_output_length.csv wt-8
49 <= 50 N2_output_length.csv wt-6
43 <= 50 N2_output_length.csv wt-5


In [22]:
# step 7: output distributive images
for output in all_output_folders:
    plot_image(output)

['N2_output_length_random_selection.csv', 'N2_output_dist_random_selection.csv']
0.463630909955
0.00313879888651
0.141706501352
0.00146126042203
